In [1]:
from arguments import ModelParams, ArgumentParser
from depth_images import calibrate_depth
from gaussian_renderer import GaussianModel
from scene import Scene
import matplotlib.pyplot as plt

parser = ArgumentParser()
dataset = ModelParams(parser)
dataset.model_path = ""
dataset.source_path = "../data/redwood_proc/00002/"
dataset.images = "images"
dataset.resolution = -1
dataset.white_background = False
dataset.eval=True
dataset.num_train_images = 5
gaussians = GaussianModel(dataset.sh_degree)
scene = Scene(dataset, gaussians,shuffle=False)

calibrate_depth(scene)

Reading camera 192/192
Converting point3d.bin to .ply, will happen only the first time you open the scene.
Loading Training Cameras
Loading Test Cameras
Number of points at initialisation :  2489


In [ ]:
scene.